In [ ]:
import os
from google.colab import drive
#mounting it to google drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

#The files must be in a called folder GPT-anvit
sys.path.append('/content/drive/MyDrive/GPT-anvit')

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm.notebook import tqdm
from model import GPTModel
from gpt_config import GPTConfig
from data_utils import train_tokenizer, load_tokenizer, load_data
from utils import generate_text, calculate_perplexity
import time


In [ ]:
#data shoudl be in a folder called data inside GPT-anvit
data_files = ["/content/drive/MyDrive/GPT-anvit/data/finance_corpus.txt"]  # your data
tokenizer_path = "tokenizer_finance.json"
if not os.path.exists(tokenizer_path):
    tokenizer = train_tokenizer(data_files, vocab_size=10000, save_path=tokenizer_path)
else:
    tokenizer = load_tokenizer(tokenizer_path)

pad_id = tokenizer.token_to_id("[PAD]")


Tokenizer trained and saved to tokenizer_finance.json


In [ ]:
block_size = 256
vocab_size = 10000
train_dataset, val_dataset = load_data(data_files, tokenizer, block_size=block_size)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = GPTConfig(vocab_size=vocab_size, max_len=1024, n_layer=6, n_head=8, n_embd=512)
model = GPTModel(config).to(device)

In [ ]:
total_start_time = time.time()

epochs = 10
learning_rate = 3e-4
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs + 1):
    start_time = time.time()

    model.train()
    total_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    end_time = time.time()
    epoch_time = end_time - start_time

    print(f"\nEpoch {epoch} completed in {epoch_time:.2f} seconds.")
    print(f"Total training loss: {total_loss:.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch} Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    perplexity = torch.exp(torch.tensor(avg_val_loss))
    print(f"Epoch {epoch} Validation Loss: {avg_val_loss:.4f}, Perplexity: {perplexity:.4f}")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total training time: {total_duration:.2f} seconds")


Epoch 1/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 1 completed in 117.71 seconds.
Total training loss: 9800.4120
Peak GPU memory used: 2523.91 MB
Epoch 1 Training Loss: 4.5018
Epoch 1 Validation Loss: 3.5928, Perplexity: 36.3344


Epoch 2/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 2 completed in 116.89 seconds.
Total training loss: 7362.6499
Peak GPU memory used: 2557.47 MB
Epoch 2 Training Loss: 3.3820
Epoch 2 Validation Loss: 3.0919, Perplexity: 22.0195


Epoch 3/10:   0%|          | 0/2177 [00:00<?, ?it/s]

...


Epoch 8 completed in 115.59 seconds.
Total training loss: 4987.0888
Peak GPU memory used: 2523.91 MB
Epoch 8 Training Loss: 2.2908
Epoch 8 Validation Loss: 2.4392, Perplexity: 11.4640


Epoch 9/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 9 completed in 115.54 seconds.
Total training loss: 4829.9110
Peak GPU memory used: 2523.91 MB
Epoch 9 Training Loss: 2.2186
Epoch 9 Validation Loss: 2.4036, Perplexity: 11.0634


Epoch 10/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 10 completed in 115.49 seconds.
Total training loss: 4692.7724
Peak GPU memory used: 2523.91 MB
Epoch 10 Training Loss: 2.1556
Epoch 10 Validation Loss: 2.3788, Perplexity: 10.7923
Total training time: 1211.23 seconds


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/GPT-anvit/finance_gpt.pth")
print("Model saved to finance_gpt.pth")

#model.load_state_dict(torch.load("finance_gpt.pth", map_location=device))


Model saved to finance_gpt.pth


In [ ]:
prompt = "Tell me which stocks are promising?"
prompt = "Write a financial analysis of promising stocks. Include:\n- A general market overview\n- Mention of specific companies (e.g. IBM)\n- Key financial stats (earnings, revenue, beta)\n- Analyst sentiment\n- Final investment suggestion\n"

generated = generate_text(model, tokenizer, prompt, max_length=500, top_k=20, device=device)
print(generated)


Wr ite a financial analysis of promising stocks . Inclu de : - A general market over view - M ention of specific companies ( e . g . IBM ) - Key financial st ats ( earnings , revenue , beta ) - Analyst sentiment - Fin al investment suggest ion and value - focused investors should be concerned with any potential impact on the long - term growth . With sustained growth and disciplined management , the company is committed to increasing the dividend annually from an increasing focus on the investor base . Additionally , the company has been actively expanding its existing portfolio and focus on acquisitions that align with consumer preferences . IBM aims to increase cost savings by increasing expenses , maintaining strong margins , improving margins and effective capital allocation . This shift is aimed at maxim izing on margin improvements . With a diversified revenue base and expanding margins , management aims to enhance profitability while maintaining disciplined capital - intensive d